In [ ]:
!wget https://raw.githubusercontent.com/dl-workshop/python/master/data.zip

In [ ]:
!unzip "/content/data.zip"

In [ ]:
from tensorflow.keras import layers, models
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras import callbacks

import sys
import os
import keras
import time
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
global class_names

class_names = ['dingo', 'hunting_dog', 'japanese_spaniel', 'mexican_hairless', 'standard_poodle']

In [ ]:
PATH = os.path.join(os.path.dirname(os.getcwd()),'/content/data')
train_dir = os.path.join(PATH, 'train')
test_dir = os.path.join(PATH, 'test')

train_image_generator = ImageDataGenerator(rescale=1./255)  # Generator for our training data	
test_image_generator = ImageDataGenerator(rescale=1./255)  # Generator for our validation data

batch_size = 1000
IMG_HEIGHT = 128
IMG_WIDTH = 128

train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size, directory=train_dir, shuffle=True, target_size=(IMG_HEIGHT, IMG_WIDTH), class_mode='binary')
test_data_gen = test_image_generator.flow_from_directory(batch_size=batch_size, directory=test_dir, target_size=(IMG_HEIGHT, IMG_WIDTH), class_mode='binary')

x_train, y_train = next(train_data_gen)
x_test, y_test = next(test_data_gen)

In [ ]:
x_train.shape

In [ ]:
x_train[0]

In [ ]:
y_train[0]

In [ ]:
y_train = keras.utils.to_categorical(y_train, 5)
y_test = keras.utils.to_categorical(y_test, 5)

In [ ]:
y_train[0]

In [ ]:
model = models.Sequential()
model.add(layers.BatchNormalization(input_shape=(128, 128, 3)))
model.add(layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(layers.Dropout(0.25))

model.add(layers.Flatten())
model.add(layers.Dense(256))
model.add(layers.Activation('elu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(5))
model.add(layers.Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss="categorical_crossentropy", 
              optimizer="adam", 
              metrics=["accuracy"])

In [ ]:
model.fit(x_train, y_train, epochs=5)

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
predictions = model.predict(x_test)

In [ ]:
np.argmax(predictions[0])

In [ ]:
class_names[np.argmax(predictions[0])]

In [ ]:
def show_image_prediction(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  
  plt.imshow(img, cmap=plt.cm.binary)
  
  predicted_label = np.argmax(predictions_array)
  predicted_label = class_names[predicted_label]
  for i, value in enumerate(true_label):
    if value > 0:
      true_label = class_names[i]

  print('Prediction:\t{} ({})'.format(predicted_label, max(predictions_array)))
  print('Target:\t\t{}'.format(true_label))

In [ ]:
i = 0
show_image_prediction(i, predictions, y_test, x_test)